# Exercise 9: 

In [1]:
from gurobipy import Model, GRB, quicksum

# Data
W_nom = 150  # MW
c_GW1 = 15  # EUR/MWh
lambda_DA = 20  # EUR/MWh
lambda_B = [0, 7, 10, 15, 24, 26, 35, 40, 43, 50]  # Balancing prices
P_W1 = [75, 125]  # Wind production levels
prob_lambda_B = 0.1
prob_P_W1 = 0.5

# Probabilities of all scenarios
scenarios = [(p, l) for p in P_W1 for l in lambda_B]
scenario_probs = {s: prob_P_W1 * prob_lambda_B for s in scenarios}

# (a) Profit Maximization
model = Model("Profit Maximization")

# Decision variable: Quantity offered in the day-ahead market
q = model.addVar(lb=0, ub=W_nom, name="q")

# Scenario-based profits
profits = {}
for (p, l) in scenarios:
    profits[(p, l)] = model.addVar(name=f"profit_{p}_{l}")

# Objective: Maximize expected profit
model.setObjective(
    quicksum(profits[s] * scenario_probs[s] for s in scenarios),
    GRB.MAXIMIZE
)

# Constraints: Calculate profits for each scenario
for (p, l) in scenarios:
    imbalance = p - q
    model.addConstr(
        profits[(p, l)] == lambda_DA * q - c_GW1 * p + l * imbalance
    )

model.optimize()

# Results for (a)
if model.Status == GRB.OPTIMAL:
    print(f"Optimal q (day-ahead offer): {q.X}")
    for s in scenarios:
        print(f"Profit in scenario {s}: {profits[s].X}")

# (b) Risk Aversion with CVaR
alpha = 0.95
beta = 0.5  # Example risk-aversion parameter
eta = model.addVar(name="eta")
z = {s: model.addVar(lb=0, name=f"z_{s}") for s in scenarios}

# Update the objective to include CVaR
model.setObjective(
    (1 - beta) * quicksum(profits[s] * scenario_probs[s] for s in scenarios)
    - beta * (eta + 1 / (1 - alpha) * quicksum(z[s] * scenario_probs[s] for s in scenarios)),
    GRB.MAXIMIZE
)

# CVaR constraints
for s in scenarios:
    model.addConstr(z[s] >= eta - profits[s])

model.optimize()

# Results for (b)
if model.Status == GRB.OPTIMAL:
    print(f"Optimal q (day-ahead offer): {q.X}")
    print(f"CVaR Value: {eta.X}")
    for s in scenarios:
        print(f"Profit in scenario {s}: {profits[s].X}")


Restricted license - for non-production use only - expires 2025-11-24
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10700T CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 20 rows, 21 columns and 40 nonzeros
Model fingerprint: 0x99a5f6ca
Coefficient statistics:
  Matrix range     [1e+00, 3e+01]
  Objective range  [5e-02, 5e-02]
  Bounds range     [2e+02, 2e+02]
  RHS range        [4e+02, 4e+03]
Presolve removed 8 rows and 8 columns
Presolve time: 0.01s

Solved in 0 iterations and 0.01 seconds (0.00 work units)
Infeasible or unbounded model
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i7-10700T CPU @ 2.00GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 8 physical cores, 16 logical processors, using up to 16 threads

Optimize a model with 40 rows, 42 colu